In [1]:
import os
import pickle
import numpy as np
import matplotlib, matplotlib.pyplot as plt
import networkx as nx
import pandas as pd
import torch
import itertools
from torch.autograd import Variable
import sklearn, sklearn.model_selection, sklearn.metrics
import numpy as np

from models.model_wrapper import WrappedModel
from data import datasets
from data.graph_wrapper import GeneInteractionGraph

/Users/martinweiss/.pyenv/versions/2.7.13/lib/python2.7/site-packages/h5py/__init__.py:36: FutureWarning: Conversion of the second argument of issubdtype from `float` to `np.floating` is deprecated. In future, it will be treated as `np.float64 == np.dtype(float).type`.
  from ._conv import register_converters as _register_converters


In [2]:
# If you don't pass in a path, the dataset will be downloaded from Academic Torrents 

#datsaet = datasets.GeneDataset(file_path='/data/lisa/data/genomics/TCGA/TCGA_tissue_ppi.hdf5')
dataset = datasets.GeneDataset(at_hash="4070a45bc7dd69584f33e86ce193a2c903f0776d")

Converting one-hot labels to integers


In [58]:
# Setup the results dictionary
results_file_name = "experiments/results/fig-5.pkl"
try:
    results = pickle.load(open(results_file_name, "r"))
    print "Loaded Checkpointed Results"
except Exception as e:
    results = pd.DataFrame(columns=['auc', 'gene', 'model', 'num_genes', 'seed', 'train_size'])
    print "Created a New Results Dictionary"


Loaded Checkpointed Results


In [5]:
path = "ae2691e4f4f068d32f83797b224eb854b27bd3ee" # Gene mania hash for Academic Torrents
#path = "/data/lisa/data/genomics/graph/pancan-tissue-graph.hdf5" # Filepath for graph file.

gene_graph = GeneInteractionGraph(path)
mapping = dict(zip(range(0, len(dataset.df.columns)), dataset.df.columns))
gene_graph.nx_graph = nx.relabel_nodes(gene_graph.nx_graph, mapping)

In [59]:

def record_result(results, gene, model, num_genes, train_size, seed, auc, results_file_name):
    experiment = {
        "gene": gene,
        "model": model.name,
        "num_genes": num_genes,
        "seed": seed,
        "train_size": train_size,
        "auc": auc
    }
    results = results.append(experiment, ignore_index=True)
    results_dir = "/".join(results_file_name.split('/')[0:-1])

    if not os.path.isdir(results_dir):
        os.makedirs(results_dir)
    pickle.dump(results, open(results_file_name, "wb"))
    return results

In [48]:
search_num_genes=[50, 100, 200, 300, 500, 1000, 2000, 4000, 8000, 16000]
test_size=1000
search_train_size=[50]
cuda=False
trials=5
search_genes = ["CEBPD", "IL5", "PABPC3", "PSMB10", "RPL13", "RPL4", "RPL5", "RPS10", "RPS3", "S100A8", "S100A9", "TOP1", "C15orf40", "RNF138", "DLGAP2", "EVI2B", "ZFP82", "MYBL2", "PSMB1", "CISD1", "HLA-B", "SAA2", "IFIT1", "RPS3A", "TP53", "TNF", "EGFR"]
models = [WrappedModel(name="GCN_lay3_chan64_emb32_dropout", cuda=cuda, num_layer=3, channels=64, embedding=32),
          WrappedModel(name="MLP_lay2_chan512_dropout", cuda=cuda, dropout=True, num_layer=2, channels=512),
          WrappedModel(name="MLP_lay2_chan512", cuda=cuda, dropout=False, num_layer=2, channels=512),
          WrappedModel(name="SLR_lambda1_l11", cuda=cuda),
          WrappedModel(name="GCN_lay20_chan32_emb32_dropout_pool", cuda=cuda, num_layer=4, channels=32, embedding=32, prepool_extralayers=5, pooling="hierarchy")  
         ]

In [60]:
# Create the set of all experiment ids and see which are left to do
model_names = [model.name for model in models]
columns = ["gene", "model", "num_genes", "train_size", "seed"]
all_exp_ids = [x for x in itertools.product(search_genes, model_names, search_num_genes, search_train_size, range(trials))]
all_exp_ids = pd.DataFrame(all_exp_ids, columns=columns)
all_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in all_exp_ids.itertuples(name=None)]
results_exp_ids = results[columns].copy()
results_exp_ids.index = ["-".join(map(str, tup[1:])) for tup in results_exp_ids.itertuples(name=None)]
intersection_ids = all_exp_ids.index.intersection(results_exp_ids.index)
todo = all_exp_ids.drop(intersection_ids)
print("todo: " + str(len(todo)))
print("done: " + str(len(results)))

todo: 6748
done: 3


In [61]:
for exp_id in todo.index:
    if len(results) % 10 == 0:
        print(len(results))
    gene, model_name, train_size, num_genes, seed = exp_id.split("-")
    seed = int(seed)
    num_genes = int(num_genes)
    train_size = int(train_size)
    model = [x for x in models if x.name == model_name][0]

    gene_expression_mean = dataset.df[gene].mean()
    dataset.labels = [1 if x > gene_expression_mean else 0 for x in dataset.df[gene]]
    X_train, X_test, y_train, y_test = sklearn.model_selection.train_test_split(dataset.df, dataset.labels, stratify=dataset.labels, train_size=train_size, test_size=test_size)

    # record the failure to the results file and continue.
    if len(set(y_train)) <= 1 or len(set(y_test)) <= 1:
        results = record_result(results, gene, model, train_size, seed, 0.5, results_file_name)
        continue

    neighbors, neighborhood = gene_graph.first_degree(gene)
    X_train = X_train[list(neighbors)].copy()
    X_test = X_test[list(neighbors)].copy()

    X_train[gene] = 1
    X_test[gene] = 1

    model.fit(X_train, y_train, adj=neighborhood)

    x_test = Variable(torch.FloatTensor(np.expand_dims(X_test.values, axis=2)), requires_grad=False).float()
    if cuda:
        x_test.cuda()
    y_hat = model.predict(x_test)[:,1].data.cpu().numpy()
    auc = sklearn.metrics.roc_auc_score(y_test, np.asarray(y_hat).flatten())

    model.best_model = None # cleanup
    results = record_result(results, gene, model, num_genes, train_size, seed, auc, results_file_name)


10
20


KeyboardInterrupt: 

In [62]:

for plot_gene in genes:

    %matplotlib inline
    plt.rcParams['figure.figsize'] = (7.5, 3.6)
    plot_train_size = 50

    subset = results["df"][(results["df"].train_size==plot_train_size) & 
                      (results["df"].gene==plot_gene) & 
                      (results["df"].num_genes!=400) &      
                      (results["df"].num_genes> 0)]


    q = subset.groupby(['model','num_genes'])['auc']

    todo = list(subset["model"].unique())
    linestyles = ['-', '-', '--', '-.', ':']
    for ls, model in enumerate(sorted(todo)):
        index = list(q.mean()[model].index)
        mean = q.mean()[model]
        stderr = q.std()[model]/np.sqrt(q.count()[model])
        displayname = model.replace("CGN","GCN")
        displayname = displayname.replace("SLR", "SNLR")
        plt.errorbar(index, mean,label=displayname, xerr=0, yerr=stderr, ls=linestyles[ls])

    plt.title("Gene Inference " + plot_gene + " (train_size:" + str(plot_train_size) +")")
    plt.ylabel("AUC")
    plt.xlabel("Number of genes")
    plt.xscale("log")
    plt.xticks(sorted(subset["num_genes"].unique()))
    formatter = matplotlib.ticker.ScalarFormatter()
    plt.gca().xaxis.set_major_formatter(formatter)

    plt.legend();
    fd = len(list(gene_graph.nx_graph.neighbors(plot_gene)))
    print fd
    if fd > 50:
        plt.axvline(fd, ymin=0.4, ymax=1.0, c="black")
        c = plt.ylim()
        plt.text(fd*1.05,c[1]-((c[1]-c[0])*0.2),'First Degree',rotation=90)



    plt.savefig("experiments/results/sgi-" + plot_gene + "-" + "train" + str(plot_train_size) + ".png", bbox_inches='tight')
    plt.show()

NameError: name 'genes' is not defined